In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import *
from torchvision import datasets
from torchvision.transforms import *

In [ ]:
# Download ISTSat1 attitude dataset
DATAFILE1 = 'data/istsat1/NoSensorFault.csv'

import numpy as np

data_tensor = torch.tensor(np.loadtxt(DATAFILE, delimiter=',')[:,1:])
data_tensor.shape

In [ ]:
# Download ISTSat1 attitude dataset

DATAFILE1 = 'data/istsat1/NoSensorFault.csv'
DATAFILE2 = 'data/istsat1/SensorFault20000_25000.csv'

import numpy as np

class ISTSat1Dataset(Dataset):
    window = 100
    stride = 50
    
    def __init__(self, filename):
        raw = np.loadtxt(filename, delimiter=',')
        self.data = torch.tensor(raw[600:])  # HACK: cutting first 6000 secs due to detumbling
        self.total = self.data.shape[0]
        self.size = 1 + int((self.total - self.window)/self.stride)

    def __len__(self):
        return self.size

    def __getitem__(self, i):
        a = i * self.stride
        b = a + self.window
        return self.data[a:b,1:5]

dataset1 = ISTSat1Dataset(DATAFILE1)
print(f"Loaded {DATAFILE1}: {len(dataset1)} data points (windows)")

dataset2 = ISTSat1Dataset(DATAFILE2)
print(f"Loaded {DATAFILE2}: {len(dataset2)} data points (windows)")

In [ ]:
# Show some examples of the dataset

(fig,ax) = plt.subplots(2, 5, constrained_layout=True)
for (i,d) in enumerate([dataset1, dataset2]):
    for j in range(5):
        for k in range(4):
            ax[i,j].plot(d[j][:,k])
        ax[i,j].set_axis_off()
plt.show()

In [ ]:
batch_size = 32

# Create data loaders.
dataloader1 = DataLoader(dataset1, batch_size=batch_size)
dataloader2 = DataLoader(dataset2, batch_size=batch_size)

# DELME
for X in dataloader1:
    print(f"Shape of X: {X.shape}")
    break

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Define model
class Autoencoder(nn.Module):
    bottleneck = 10
    
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.unflatten = nn.Unflatten(1, (100,4))
        self.encoder = nn.Sequential(
            nn.Linear(100*4, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, self.bottleneck)
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.bottleneck, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 100*4),
        )

    def encode(self, x):
        x = self.flatten(x)
        coded = self.encoder(x)
        return coded

    def decode(self, x):
        x = self.decoder(x)
        decoded = self.unflatten(x)
        return decoded

    def forward(self, x):
        return self.decode( self.encode(x) )

In [ ]:
# TO BE UPDATED

class ConvAutoencoderMNIST(nn.Module):
    bottleneck = 10
    
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=2, stride=2),
            nn.Sigmoid() # Output should be in the range [0, 1]
        )

    def encode(self, x):
        coded = self.encoder(x)
        return coded

    def decode(self, x):
        decoded = self.decoder(x)
        return decoded

    def forward(self, x):
        return self.decode( self.encode(x) )

In [ ]:
#model = ClassicAutoencoderMNIST().to(device)
model = ConvAutoencoderMNIST().to(device)
print(model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, X)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss_fn, optimizer)
print("Done!")

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

In [ ]:
# Show some examples of the dataset

model.eval()
(fig,ax) = plt.subplots(3, 10, constrained_layout=True)
ds = full_dataset
for j in range(10):
    x, _ = ds[j]
    x = x[None,:].to(device)
    with torch.no_grad():
        z = model.encode(x)
        print(z.shape)
        y = model.decode(z)
    
    ax[0,j].imshow(ds[j][0][0], cmap="gray")
    ax[0,j].set_axis_off()
    ax[1,j].imshow(z[None,0].cpu(), cmap="gray")
    ax[1,j].set_axis_off()
    ax[2,j].imshow(y[0][0].cpu(), cmap="gray")
    ax[2,j].set_axis_off()
    
plt.axis('off')
plt.show()

In [ ]:
# determine mean and covariance of encoded data, and perform spectral decompositon

all_data = torch.stack([x for (x,_) in full_dataset])
all_data = all_data.to(device)

model.eval()
with torch.no_grad():
    out = model.encode(all_data)
    out = out.cpu()

u = torch.mean(out,0)
S = torch.cov(out.T)

(L,Q)=torch.linalg.eigh(S)

In [ ]:
# generate some images from random encoded data

model.eval()
(fig,ax) = plt.subplots(2, 10, constrained_layout=True)
for j in range(10):
    z = torch.randn(10)
    #z = u + torch.randn(10)
    #z = u + Q@torch.sqrt(L)*torch.randn(10)
    z = z[None,:].to(device)

    with torch.no_grad():
        y = model.decode(z)
    
    ax[0,j].imshow(z[None,0].cpu(), cmap="gray")
    ax[0,j].set_axis_off()
    ax[1,j].imshow(y[0][0].cpu(), cmap="gray")
    ax[1,j].set_axis_off()
    
plt.axis('off')
plt.show()

In [ ]:
# generate some images along 2 principal axes

model.eval()
(fig,ax) = plt.subplots(11, 11, constrained_layout=True)
for i in range(11):
    for j in range(11):
        z = u + (i-5)*torch.sqrt(L[-2])*Q[:,-2] \
              + (j-5)*torch.sqrt(L[-1])*Q[:,-1]
        z = z[None,:].to(device)
    
        with torch.no_grad():
            y = model.decode(z)
        
        ax[i,j].imshow(y[0][0].cpu(), cmap="gray")
        ax[i,j].set_axis_off()
    
plt.axis('off')
plt.show()

In [ ]:
# generate some images along the 10 highest principal values

model.eval()
(fig,ax) = plt.subplots(10, 11, constrained_layout=True)
for i in range(10):
    for j in range(11):
        z = u + (j-5)*torch.sqrt(L[-1-i])*Q[:,-1-i]
        z = z[None,:].to(device)
    
        with torch.no_grad():
            y = model.decode(z)
        
        ax[i,j].imshow(y[0][0].cpu(), cmap="gray")
        ax[i,j].set_axis_off()
    
plt.axis('off')
plt.show()